In [1]:
from google.colab import files
upload=files.upload()

Saving spam.csv to spam.csv


In [2]:
import pandas as pd
import numpy as np

In [26]:
data=pd.read_csv("/content/spam.csv")

In [27]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
data.isnull().sum()

,0
Category,0
Message,0


In [29]:
#data['Category'] = data['Category'].map({'spam':1,'ham':0})
#data.head()

In [30]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = text.lower()
    tokens = nltk.word_tokenize(text) # Tokenization
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words] # Remove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens] # Lemmatization
    return " ".join(tokens)

data['Message'] = data['Message'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
print(data[['Message']].head())

                                             Message
0  go jurong point crazy available bugis n great ...
1                            ok lar joking wif u oni
2  free entry 2 wkly comp win fa cup final tkts 2...
3                u dun say early hor u c already say
4           nah dont think go usf life around though


In [32]:
data.head()

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think go usf life around though


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
# Encode labels
data['Category'] = data['Category'].map({'ham': 0, 'spam': 1})

In [41]:
data.head()

,Category,Message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say early hor u c already say
4,0,nah dont think go usf life around though


In [42]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(data['Message'], data['Category'], test_size=0.2, random_state=42)

In [43]:
# Vectorize text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [46]:
# Train the model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [47]:
# Make predictions
y_pred = model.predict(X_test_tfidf)

In [48]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.967713004484305
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.76      0.86       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115

Confusion Matrix:
 [[966   0]
 [ 36 113]]


In [49]:
import joblib

In [50]:
# Save the model and vectorizer
joblib.dump(model, 'spam_detection_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

# Loading the model and vectorizer for prediction
model_loaded = joblib.load('spam_detection_model.pkl')
vectorizer_loaded = joblib.load('vectorizer.pkl')

In [51]:
# Function to predict if a message is spam or not
def predict_message(message):
    processed_message = preprocess_text(message)
    message_tfidf = vectorizer_loaded.transform([processed_message])
    prediction = model_loaded.predict(message_tfidf)
    return "spam" if prediction[0] == 1 else "ham"

In [56]:
# Test prediction
new_message =input("Enter the Message: ")
prediction = predict_message(new_message)
print(f"The message is: {prediction}")

Enter the Message: The Phony Sweepstakes Winner: “Congratulations! ...
The message is: spam
